<a href="https://colab.research.google.com/github/ehsan-lari/pyro101/blob/main/pyro_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bayesian Neural Networks